In [65]:
import numpy as np
import pandas as pd
from scipy.special import expit
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style



style.use("fivethirtyeight")

np.random.seed(123)
n = 100

In [66]:
import sys
sys.path.append('../scripts/')

from data_cleaning import DataCleaning
from data_extraction import DataExtraction
dclean = DataCleaning()
dextract = DataExtraction()


In [67]:
trip_df = dextract.load_data("../data/nb.csv")
trip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536020 entries, 0 to 536019
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Trip ID           536020 non-null  int64 
 1   Trip Origin       536020 non-null  object
 2   Trip Destination  536020 non-null  object
 3   Trip Start Time   534369 non-null  object
 4   Trip End Time     536019 non-null  object
dtypes: int64(1), object(4)
memory usage: 20.4+ MB


- path distance-average - api
- [x] displacement
- [x] duration  
- year/season/month/weekdays
- holiday-api -  
- distance after & before holiday
- weather - location & date - api
- traffic - api 

- duration vs holiday
- rejection rate / acceptance rate

In [68]:
miss_info = trip_df.isna().sum()
miss_info

Trip ID                0
Trip Origin            0
Trip Destination       0
Trip Start Time     1651
Trip End Time          1
dtype: int64

In [69]:
trip_df.rename(columns = {'Trip ID':'order_id',
                          'Trip Origin':"trip_origin",
                          'Trip Destination':'trip_destination',
                          'Trip Start Time':'trip_Start_time',
                          'Trip End Time':'trip_end_time'}, inplace = True)

In [70]:
# trip_df = get_lat_long(trip_df,['trip_origin','trip_destination'])

In [71]:
trip_df.head()

,order_id,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


There are 1651 Trip which has no record of trip start time but has trip end time. Why the is a record for trip end time but not for trip start time? Do the driver register their end time only to do some tricks or is there system error for those sptecific trip without trip start time? 

In [72]:
order_df = dextract.load_data("../data/driver_locations_during_request.csv")
order_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id             1557740 non-null  int64  
 1   order_id       1557740 non-null  int64  
 2   driver_id      1557740 non-null  int64  
 3   driver_action  1557740 non-null  object 
 4   lat            1557740 non-null  float64
 5   lng            1557740 non-null  float64
 6   created_at     0 non-null        float64
 7   updated_at     0 non-null        float64
dtypes: float64(4), int64(3), object(1)
memory usage: 95.1+ MB


- driver_origin - order_location

- distance vs rejection

In [73]:
order_df['order_origin'] = [str(x)+","+str(y) for x,y in zip(order_df.lat, order_df.lng)] 
                                        

In [74]:
order_df.head()

,id,order_id,driver_id,driver_action,lat,lng,created_at,updated_at,order_origin
0,1,392001,243828,accepted,6.602207,3.270465,NaN,NaN,"6.6022066,3.2704649"
1,2,392001,243588,rejected,6.592097,3.287445,NaN,NaN,"6.5920972,3.2874447"
2,3,392001,243830,rejected,6.596133,3.281784,NaN,NaN,"6.5961334,3.2817841"
3,4,392001,243539,rejected,6.596142,3.280526,NaN,NaN,"6.5961416,3.2805263"
4,5,392001,171653,rejected,6.609232,3.288800,NaN,NaN,"6.6092317,3.2887999"


## Merge the tables
Use order_id and Trip ID columns to merge as the two columns are similar

In [75]:
df = order_df.merge(trip_df, how='inner', on='order_id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1557740 entries, 0 to 1557739
Data columns (total 13 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557740 non-null  int64  
 1   order_id          1557740 non-null  int64  
 2   driver_id         1557740 non-null  int64  
 3   driver_action     1557740 non-null  object 
 4   lat               1557740 non-null  float64
 5   lng               1557740 non-null  float64
 6   created_at        0 non-null        float64
 7   updated_at        0 non-null        float64
 8   order_origin      1557740 non-null  object 
 9   trip_origin       1557740 non-null  object 
 10  trip_destination  1557740 non-null  object 
 11  trip_Start_time   1555718 non-null  object 
 12  trip_end_time     1557740 non-null  object 
dtypes: float64(4), int64(3), object(6)
memory usage: 166.4+ MB


In [76]:
df.isna().sum()

id                        0
order_id                  0
driver_id                 0
driver_action             0
lat                       0
lng                       0
created_at          1557740
updated_at          1557740
order_origin              0
trip_origin               0
trip_destination          0
trip_Start_time        2022
trip_end_time             0
dtype: int64

## Clean DF
Drop 'created_at' and 'updated_at' -- all the entry is empty.

In [77]:

clean_df = dclean.drop_columns(df,['created_at', 'updated_at'])
clean_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1557740 entries, 0 to 1557739
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   id                1557740 non-null  int64  
 1   order_id          1557740 non-null  int64  
 2   driver_id         1557740 non-null  int64  
 3   driver_action     1557740 non-null  object 
 4   lat               1557740 non-null  float64
 5   lng               1557740 non-null  float64
 6   order_origin      1557740 non-null  object 
 7   trip_origin       1557740 non-null  object 
 8   trip_destination  1557740 non-null  object 
 9   trip_Start_time   1555718 non-null  object 
 10  trip_end_time     1557740 non-null  object 
dtypes: float64(2), int64(3), object(6)
memory usage: 142.6+ MB


In [78]:
clean_df.head()

,id,order_id,driver_id,driver_action,lat,lng,order_origin,trip_origin,trip_destination,trip_Start_time,trip_end_time
0,1,392001,243828,accepted,6.602207,3.270465,"6.6022066,3.2704649","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
1,2,392001,243588,rejected,6.592097,3.287445,"6.5920972,3.2874447","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
2,3,392001,243830,rejected,6.596133,3.281784,"6.5961334,3.2817841","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
3,4,392001,243539,rejected,6.596142,3.280526,"6.5961416,3.2805263","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36
4,5,392001,171653,rejected,6.609232,3.288800,"6.6092317,3.2887999","6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [79]:
clean_df.to_csv('../data/clean_data.csv',index=False)